# Assignment 1: Named Entity Recognition and Entity Linking Group Project

Authors:  
Alberto de los Ríos

## Imports

In [ ]:
import pandas as pd
import os


In [ ]:
# It is recommended to start with general import statements
#from utility_functions import *

## Load data

This section should load the raw dataset for the task.  
Remember to use relative paths to load any files in the notebook.

In [ ]:
url_1177 = "hf://datasets/community-datasets/swedish_medical_ner/1177/train-00000-of-00001.parquet"
url_lt = "hf://datasets/community-datasets/swedish_medical_ner/lt/train-00000-of-00001.parquet"
url_wiki = "hf://datasets/community-datasets/swedish_medical_ner/wiki/train-00000-of-00001.parquet"

df_1177 = pd.read_parquet(url_1177)
#df_lt = pd.read_parquet(url_lt)
#df_wiki = pd.read_parquet(url_wiki)

os.makedirs("raw_data", exist_ok=True)
df_1177.to_parquet("raw_data/1177_train.parquet", engine="pyarrow")
#df_lt.to_parquet("raw_data/lt_train.parquet", engine="pyarrow")
#df_wiki.to_parquet("raw_data/wiki_train.parquet", engine="pyarrow")


## Task 1: LLMs for NER Survey

This survey will be evalutated based on the following metrics and dimensions:

- Language Capability (Swedish proficiency)
- Biomedical Knowledge (Domain relevance)
- Computational Efficiency (Training/inference costs)
- Performance Metrics (NER-specific scores)
- Ontology Compatibility (Linking to ICD/ICF/LOINC)

PONER UN POCO DE HISTORIA DE BERT creado por los researches the Google

For this first task we will only use the 1177 Vårdguiden dataset since it is the lightest with only 927 sentences.

# LLM in Swedish

Advantages:
1. Authenticity - Maintains original clinical nuance and terminology
2. No translation errors - Avoids introducing errors from machine translation
3. Proper noun handling - Swedish patient/place names and untranslatable terms remain intact
4. Future applicability - Model will work natively with Swedish EHR systems
5. Matching Swedish ontologies - Direct alignment with Swedish medical coding systems

Limitations:
1. Limited models - Fewer Swedish biomedical LLMs available:
- KB/bert-base-swedish-cased: [Swedish BERT models for NER](https://huggingface.co/KB/bert-base-swedish-cased)
- [RoBERTa large](https://huggingface.co/AI-Sweden-Models/roberta-large-1160k)
- Swedish GPT models with limited NER capacity: [AI Sweden Model Hub](https://huggingface.co/AI-Sweden-Models)
2. Smaller datasets - The Swedish medical NER dataset has only ~6,000 annotated entries
3. Debugging difficulty - Hard to verify annotations/errors without Swedish knowledge
4. Resource scarcity - Few Swedish stopword lists, tokenizers, etc.


# LLM in English

- [Swedish BERT models for NER](https://huggingface.co/KB/bert-base-swedish-cased)
- [RoBERTa large](https://huggingface.co/AI-Sweden-Models/roberta-large-1160k)
- [AI Sweden Model Hub](https://huggingface.co/AI-Sweden-Models)


Advantages:
1. Model availability - Access powerful English biomedical models:
- [BioBERT](https://github.com/naver/biobert-pretrained?tab=readme-ov-file)
- [ClinicalBERT](https://huggingface.co/medicalai/ClinicalBERT)
- [now BiomedBERT, previously known as PubMedBERT](https://huggingface.co/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext)
2. Larger datasets - Can augment with English medical NER datasets (~20+ available)
3. Easier debugging - You can understand the text for error analysis
4. More tutorials - Abundant English NLP examples
5. Ontology linking - English ontologies (ICD-10 English) have more community support

Limitations:
1. Translation errors - Clinical terms often mistranslated:
2. Back-translation complexity - Need to map English predictions back to Swedish text
3. Loss of context - Swedish compound words get split unnaturally
4. Ontology mismatch - Swedish medical codes don't align perfectly with English
Added pipeline complexity - Requires translation component

- [BioBERT vs PubMedBERT](https://medium.com/@EleventhHourEnthusiast/model-comparison-biobert-vs-pubmedbert-8c2d78178d10)
- [Domain-Specific Language Model Pretraining for Biomedical Natural Language Processing](https://dl.acm.org/doi/10.1145/3458754)


# Multilingual LLM

- [mBERT](https://huggingface.co/google-bert/bert-base-multilingual-cased), 2018
- [XLM-RoBERTa](https://huggingface.co/FacebookAI/xlm-roberta-base), 2019


Advantages:
1. Cross-Lingual Knowledge Transfer: Leverages patterns from high-resource languages (e.g., German medical terms help Swedish). Fine-tune on Swedish data but benefit from pretraining on multilingual medical corpora.
2. Handling Code-Switching: No need for manual language detection.
3. Robust Tokenization: SentencePiece (used in XLM-R) handles Swedish compounds better than WordPiece (mBERT):
4. Future-Proofing: One model can support other languages (e.g., adding Norwegian EHRs later).

Limitations:
1. Translation errors - Clinical terms often mistranslated:
2. Back-translation complexity - Need to map English predictions back to Swedish text
3. Loss of context - Swedish compound words get split unnaturally
4. Ontology mismatch - Swedish medical codes don't align perfectly with English
Added pipeline complexity - Requires translation component

- [BioBERT vs PubMedBERT](https://medium.com/@EleventhHourEnthusiast/model-comparison-biobert-vs-pubmedbert-8c2d78178d10)
- [Domain-Specific Language Model Pretraining for Biomedical Natural Language Processing](https://dl.acm.org/doi/10.1145/3458754)


In [ ]:
# Always use comments in the code to document specific steps

## Task 2: This is the title of task 2

This section should contain the solution of task 2.

## Results and Discussion

This section should contain:
- Results.
- Summary of best model performance:
    - Name of best model file as saved in /models.
    - Relevant scores such as: accuracy, precision, recall, F1-score, etc.
- Key discussion points.

In [ ]:
# Always use comments in the code to document specific steps